In [1]:
import numpy as np
from PIL import Image

DATASET = 'mnist'
TEST_DATA_FILENAME = 't10k-images-idx3-ubyte'
TEST_LABELS_FILENAME = 't10k-labels-idx1-ubyte'
TRAIN_DATA_FILENAME = 'train-images-idx3-ubyte'
TRAIN_LABELS_FILENAME = 'train-labels-idx1-ubyte'


def bytes_to_int(byte_data):
    return int.from_bytes(byte_data, 'big')


def read_images(filename, n_max_images=None):
    images = []
    with open(filename, 'rb') as f:
        _ = f.read(4)  # magic number
        n_images = bytes_to_int(f.read(4))
        if n_max_images:
            n_images = n_max_images
        n_rows = bytes_to_int(f.read(4))
        n_columns = bytes_to_int(f.read(4))
        for image_idx in range(n_images):
            image = []
            for row_idx in range(n_rows):
                row = []
                for col_idx in range(n_columns):
                    pixel = bytes_to_int(f.read(1))  # Convert bytes to int
                    row.append(pixel)
                image.append(row)
            images.append(image)
    return images


def read_labels(filename, n_max_labels=None):
    labels = []
    with open(filename, 'rb') as f:
        _ = f.read(4)  # magic number
        n_labels = bytes_to_int(f.read(4))
        if n_max_labels:
            n_labels = n_max_labels
        for label_idx in range(n_labels):
            label = bytes_to_int(f.read(1))
            labels.append(label)
    return labels

def extract_features(X):
    return [flatten_list(sample) for sample in X]

def flatten_list(l):
    return [pixel for sublist in l for pixel in sublist]


def dist(x, y):
    return sum(
        [
            (x_i - y_i) ** 2
            for x_i, y_i in zip(x, y)
        ]
    ) ** 0.5




def get_training_distances_for_test_sample(X_train, test_sample):
    return [dist(train_sample, test_sample) for train_sample in X_train]


def get_most_frequent_element(l):
    return max(l, key=l.count)


def knn(X_train, y_train, X_test, k=3):
    y_pred = []
    for test_sample in X_test:
        training_distances = get_training_distances_for_test_sample(
            X_train, test_sample
        )
        sorted_distance_indices = [
            pair[0]
            for pair in sorted(
                enumerate(training_distances),
                key=lambda x: x[1]
            )
        ]
        candidates = [
            y_train[idx]
            for idx in sorted_distance_indices[:k]
        ]
        top_candidate = get_most_frequent_element(candidates)
        y_pred.append(top_candidate)
    return y_pred


def read_png_image(filename):
    with Image.open(filename) as img:
        img_array = np.array(img)
    return img_array.tolist()


def preprocess_png_image(png_image_filename):
    img_array = read_png_image(png_image_filename)
    test_sample = [pixel for row in img_array for pixel in row]
    return [test_sample]


def main():
    n_train = 10000
    k = 7
    print(f'Dataset: {DATASET}')
    print(f'n_train: {n_train}')
    print(f'k: {k}')

    # Adjust the path to your PNG image
    png_image_filename = './our_test.png'
    X_train = read_images(TRAIN_DATA_FILENAME, n_train)
    y_train = read_labels(TRAIN_LABELS_FILENAME, n_train)
    X_test = preprocess_png_image(png_image_filename)
    y_test = [0]  # Dummy label for the test sample (not used in prediction)

    X_train = extract_features(X_train)
    X_test = extract_features(X_test)

    y_pred = knn(X_train, y_train, X_test, k)

    print(f'Predicted label: {y_pred[0]}')


if __name__ == '__main__':
    main()

Dataset: mnist
n_train: 10000
k: 7
Predicted label: 2
